# Actor-Critic

Теорема о градиенте стратегии связывает градиент целевой функции  и градиент самой стратегии:

$$\nabla_\theta J(\theta) = \mathbb{E}_\pi [Q^\pi(s, a) \nabla_\theta \ln \pi_\theta(a \vert s)]$$

Встает вопрос, как оценить $Q^\pi(s, a)$? В чистом policy-based алгоритме REINFORCE используется отдача $G_t$, полученная методом Монте-Карло в качестве несмещенной оценки $Q^\pi(s, a)$. В Actor-Critic же предлагается отдельно обучать нейронную сеть Q-функции — критика.

Актор-критиком часто называют обобщенный фреймворк (подход), нежели какой-то конкретный алгоритм. Как подход актор-критик не указывает, каким конкретно [policy gradient] методом обучается актор и каким [value based] методом обучается критик. Таким образом актор-критик задает целое [семейство](https://proceedings.neurips.cc/paper_files/paper/1999/file/6449f44a102fde848669bdd9eb6b76fa-Paper.pdf) различных алгоритмов. Рекомендую в качестве шпаргалки использовать упомянутый в тетрадке с REINFORCE [пост из блога Lilian Weng](https://lilianweng.github.io/posts/2018-04-08-policy-gradient/), посвященный наиболее популярным алгоритмам семейства актор-критиков

В данной тетрадке познакомимся с наиболее простым вариантом актор-критика, который так и называют Actor-Critic:

In [32]:
# Cтавим нужные зависимости, если это колаб
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !pip -q install "gymnasium[classic-control, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3

In [33]:
from collections import deque

import gymnasium as gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.distributions import Categorical

%matplotlib inline

In [34]:
env = gym.make("CartPole-v1")
env.reset()

print(f'{env.observation_space=}')
print(f'{env.action_space=}')

n_actions = env.action_space.n
state_dim = env.observation_space.shape
print(f'Action_space: {n_actions} | State_space: {env.observation_space.shape}')

env.observation_space=Box([-4.8               -inf -0.41887903        -inf], [4.8               inf 0.41887903        inf], (4,), float32)
env.action_space=Discrete(2)
Action_space: 2 | State_space: (4,)


(1 балл)

In [37]:
def to_tensor(x, dtype=np.float32):
    if isinstance(x, torch.Tensor):
        return x
    x = np.asarray(x, dtype=dtype)
    x = torch.from_numpy(x)
    return x

def symlog(x):
    """Compute symlog values for a vector `x`. It's an inverse operation for symexp."""
    return torch.sign(x) * torch.log(torch.abs(x) + 1)

def symexp(x):
    """Compute symexp values for a vector `x`. It's an inverse operation for symlog."""
    return torch.sign(x) * (torch.exp(torch.abs(x)) - 1.0)


class SymExpModule(nn.Module):
    def forward(self, x):
        return symexp(x)

def select_action_eps_greedy(Q, state, epsilon):
    """Выбирает действие epsilon-жадно."""
    if not isinstance(state, torch.Tensor):
        state = torch.tensor(state, dtype=torch.float32)
    Q_s = Q(state).detach().numpy()

    if np.random.random() < epsilon:
        action = env.action_space.sample()
    else:
        action = np.argmax(Q_s)
    action = int(action)
    return action

def sample_batch(replay_buffer, n_samples):
    # sample randomly `n_samples` samples from replay buffer
    # and split an array of samples into arrays: states, actions, rewards, next_actions, terminateds
    sampled_indices = np.random.choice(len(replay_buffer), n_samples, replace=True)
    samples = [replay_buffer[i] for i in sampled_indices]
    states, actions, rewards, next_states, terminateds = zip(*samples)

    return np.array(states), np.array(actions), np.array(rewards), np.array(next_states), np.array(terminateds)

## Shared-body Actor-Critic

Актор и критик могут обучаться в разных режимах — актор только on-policy (шаг обучения на текущей собранной подтраектории), а критик on-policy или off-policy (шаг обучения на текущей подтраектории или на батче из replay buffer). Это с одной стороны привносит гибкость в обучение, с другой — усложняет его.

Если актор и критик оба обучаются on-policy, то имеет смысл объединить их сетки в одну и делать общий шаг обратного распространения ошибки. Однако, если они обучаются в разных режимах (и с разной частотой обновления), то велика вероятность, что их шаги обучения могут начать конфликтовать в случае общего тела — для такого варианта намного предпочтительнее разделить их на разные подсети (либо аккуратно настраивать гиперпарметры, чтобы стабилизировать обучение). В целом, рекомендуется использовать общий энкодер наблюдений, а далее как можно скорее разделять головы.

Сделаем реализацию актор-критика с общим телом и с on-policy вариантом обучения.

In [38]:
class ActorBatch:
    def __init__(self):
        self.logprobs = []
        self.q_values = []

    def append(self, log_prob, q_value):
        self.logprobs.append(log_prob)
        self.q_values.append(q_value)

    def clear(self):
        self.logprobs.clear()
        self.q_values.clear()

(3 балла)

In [55]:
class ActorCriticModel(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super().__init__()

        # Инициализируйте сеть агента с двумя головами: softmax-актора и линейного критика
        # self.net, self.actor_head, self.critic_head =

        layers = []
        last_dim = input_dim
        for h in hidden_dims:
            layers.append(nn.Linear(last_dim, h))
            layers.append(nn.ReLU())
            last_dim = h
        self.net = nn.Sequential(*layers)

        self.actor_head = nn.Linear(last_dim, output_dim)
        self.critic_head = nn.Linear(last_dim, output_dim)
        ##############################

    def forward(self, state):
        # Вычислите выбранное действие, логарифм вероятности его выбора и соответствующее значение Q-функции
        if state.dim() == 1:
            state = state.unsqueeze(0)
        x = self.net(state)
        logits = self.actor_head(x)
        probs = torch.softmax(logits, dim=-1)
        dist = torch.distributions.Categorical(probs)
        action = dist.sample()
        log_prob = dist.log_prob(action)
        q_values = self.critic_head(x)
        Q_s_a = q_values.gather(1, action.unsqueeze(1)).squeeze(1)
        return action.squeeze(0), log_prob.squeeze(0), Q_s_a.squeeze(0)

    def evaluate(self, state):
        # Вычислите значения Q-функции для данного состояния

        x = self.net(state)
        q_values = self.critic_head(x)
        return q_values

(6 баллов)

In [69]:
class ActorCriticAgent:
    def __init__(self, state_dim, action_dim, hidden_dims, lr, gamma, critic_rb_size):
        self.lr = lr
        self.gamma = gamma

        # Инициализируйте модель актор-критика и SGD оптимизатор (например, `torch.optim.Adam)`)
        ####### Здесь ваш код ########
        self.actor = nn.Sequential(
            nn.Linear(state_dim, hidden_dims[0]),
            nn.ReLU(),
            *[nn.Sequential(nn.Linear(hidden_dims[i], hidden_dims[i+1]), nn.ReLU()) for i in range(len(hidden_dims)-1)],
            nn.Linear(hidden_dims[-1], action_dim),
            nn.Softmax(dim=-1)
        )

        self.critic = nn.Sequential(
            nn.Linear(state_dim, hidden_dims[0]),
            nn.ReLU(),
            *[nn.Sequential(nn.Linear(hidden_dims[i], hidden_dims[i+1]), nn.ReLU()) for i in range(len(hidden_dims)-1)],
            nn.Linear(hidden_dims[-1], action_dim)
        )

        self.opt_actor = torch.optim.Adam(self.actor.parameters(), lr=lr)
        self.opt_critic = torch.optim.Adam(self.critic.parameters(), lr=lr)
        ##############################


        self.actor_batch = ActorBatch()
        self.critic_rb = deque(maxlen=critic_rb_size)

    def act(self, state):
        # Произведите выбор действия и сохраните необходимые данные в батч для последующего обучения
        # Не забудьте сделать q_value.detach()
        # self.actor_batch.append(..)
        if not isinstance(state, torch.Tensor):
            state = torch.tensor(state, dtype=torch.float32)
        probs = self.actor(state)
        dist = Categorical(probs)
        action = dist.sample()
        log_prob = dist.log_prob(action)
        q_value = self.critic(state)[action]
        self.actor_batch.append(log_prob, q_value.detach())

        return action.item()
    def append_to_replay_buffer(self, s, a, r, next_s, terminated):
        # Добавьте новый экземпляр данных в память прецедентов.
        self.critic_rb.append((s, a, r, next_s, terminated))
    def evaluate(self, state):
       return self.critic(state)

    def update(self, rollout_size, critic_batch_size, critic_updates_per_actor, global_step):
        if len(self.actor_batch.q_values) < rollout_size:
            return

        self.opt_actor.zero_grad()
        self.opt_critic.zero_grad()

        loss_c = self.update_critic(critic_batch_size, critic_updates_per_actor)
        if global_step > 10_000:
          loss_a = self.update_actor()
          (loss_a + loss_c).backward()
        else:
          loss_c.backward()

        self.opt_actor.step()
        self.opt_critic.step()

        self.actor_batch.clear()
        self.critic_rb.clear()

    def update_actor(self):
        Q_s_a = torch.stack(self.actor_batch.q_values).squeeze(-1)
        logprobs = torch.stack(self.actor_batch.logprobs)

        # Реализуйте шаг обновления актора — вычислите ошибку `loss` и произведите шаг обновления градиентным спуском.
        Q_s_a = to_tensor(self.actor_batch.q_values).detach()
        logprobs = torch.stack(self.actor_batch.logprobs)
        loss = -(Q_s_a * logprobs).mean()
        return loss
    def update_critic(self, batch_size, n_updates=1):
        # Реализуйте n_updates шагов обучения критика.
        loss = torch.tensor(0.0)
        for _ in range(n_updates):
          states, actions, rewards, next_states, terminateds = sample_batch(self.critic_rb, batch_size)
          loss = self.compute_td_loss(states, actions, rewards, next_states, terminateds)
        return loss
    def compute_td_loss(
        self, states, actions, rewards, next_states, terminateds, regularizer=0.1
    ):
        # переводим входные данные в тензоры
        s = to_tensor(states)                     # shape: [batch_size, state_size]
        a = to_tensor(actions, int).long()        # shape: [batch_size]
        r = to_tensor(rewards)                    # shape: [batch_size]
        s_next = to_tensor(next_states)           # shape: [batch_size, state_size]
        term = to_tensor(terminateds, bool)       # shape: [batch_size]


        # получаем Q[s, a] для выбранных действий в текущих состояниях (для каждого примера из батча)
        # Q_s_a = ...
        Q_s_a = self.critic(s).gather(1, a.unsqueeze(1)).squeeze(1)

        # получаем Q[s_next, *] — значения полезности всех действий в следующих состояниях
        # Q_sn = ...,
        # а затем вычисляем V*[s_next] — оптимальные значения полезности следующих состояний
        # V_sn = ...
        Q_sn = self.critic(s_next)
        V_sn = Q_sn.max(1)[0]

        # вычисляем TD target и далее TD error
        # target = ...
        # td_error = ...
        target = r + self.gamma * V_sn * (~term)
        td_error = Q_s_a - target.detach()

        # MSE loss для минимизации
        loss = torch.mean(td_error ** 2)
        # добавляем регуляризацию на значения Q
        loss += regularizer * Q_s_a.mean()
        return loss

In [70]:
def run_actor_critic(
        env_name="CartPole-v1",
        hidden_dims=(128, 128), lr=5e-4,
        total_max_steps=200_000,
        train_schedule=16, replay_buffer_size=50000, batch_size=64, critic_updates_per_actor=10,
        eval_schedule=1000, smooth_ret_window=10, success_ret=200.
):
    env = gym.make(env_name)
    episode_return_history = deque(maxlen=smooth_ret_window)

    agent = ActorCriticAgent(
        state_dim=env.observation_space.shape[0], action_dim=env.action_space.n, hidden_dims=hidden_dims,
        lr=lr, gamma=.995, critic_rb_size=replay_buffer_size
    )

    s, _ = env.reset()
    done, episode_return = False, 0.
    eval = False

    for global_step in range(1, total_max_steps+1):
        a = agent.act(s)
        s_next, r, terminated, truncated, _ = env.step(a)
        episode_return += r
        done = terminated or truncated

        # train step
        agent.append_to_replay_buffer(s, a, r, s_next, terminated)
        agent.update(train_schedule, batch_size, critic_updates_per_actor, global_step)

        # evaluate
        if global_step % eval_schedule == 0:
            eval = True

        s = s_next
        if done:
            if eval:
                episode_return_history.append(episode_return)
                avg_return = np.mean(episode_return_history)
                print(f'{global_step=} | {avg_return=:.3f}')
                if avg_return >= success_ret:
                    print('Решено!')
                    break

            s, _ = env.reset()
            done, episode_return = False, 0.
            eval = False

run_actor_critic(eval_schedule=2000, total_max_steps=100_000)

global_step=2021 | avg_return=26.000
global_step=4011 | avg_return=35.000
global_step=6005 | avg_return=28.333
global_step=8019 | avg_return=27.000
global_step=10009 | avg_return=28.600
global_step=12007 | avg_return=28.500
global_step=14013 | avg_return=36.286
global_step=16007 | avg_return=34.125
global_step=18082 | avg_return=40.222
global_step=20012 | avg_return=39.400
global_step=22000 | avg_return=38.900
global_step=24009 | avg_return=38.500
global_step=26011 | avg_return=39.000
global_step=28062 | avg_return=44.800
global_step=30017 | avg_return=44.300
global_step=32038 | avg_return=46.400
global_step=34020 | avg_return=42.100
global_step=36045 | avg_return=45.400
global_step=38009 | avg_return=40.900
global_step=40021 | avg_return=40.700
global_step=42041 | avg_return=45.500
global_step=44052 | avg_return=48.500
global_step=46016 | avg_return=53.200
global_step=48016 | avg_return=51.600
global_step=50029 | avg_return=54.600
global_step=52008 | avg_return=53.800
global_step=5403

In [71]:
def run_actor_critic(
        env_name="CartPole-v1",
        hidden_dims=(128, 128), lr=5e-4,
        total_max_steps=200_000,
        train_schedule=16, replay_buffer_size=50000, batch_size=64, critic_updates_per_actor=10,
        eval_schedule=1000, smooth_ret_window=10, success_ret=200.
):
    env = gym.make(env_name)
    episode_return_history = deque(maxlen=smooth_ret_window)

    agent = ActorCriticAgent(
        state_dim=env.observation_space.shape[0], action_dim=env.action_space.n, hidden_dims=hidden_dims,
        lr=lr, gamma=.995, critic_rb_size=replay_buffer_size
    )

    s, _ = env.reset()
    done, episode_return = False, 0.
    eval = False

    for global_step in range(1, total_max_steps+1):
        a = agent.act(s)
        s_next, r, terminated, truncated, _ = env.step(a)
        episode_return += r
        done = terminated or truncated

        # train step
        agent.append_to_replay_buffer(s, a, r, s_next, terminated)
        agent.update(train_schedule, batch_size, critic_updates_per_actor, global_step)

        # evaluate
        if global_step % eval_schedule == 0:
            eval = True

        s = s_next
        if done:
            if eval:
                episode_return_history.append(episode_return)
                avg_return = np.mean(episode_return_history)
                print(f'{global_step=} | {avg_return=:.3f}')
                if avg_return >= success_ret:
                    print('Решено!')
                    break

            s, _ = env.reset()
            done, episode_return = False, 0.
            eval = False

run_actor_critic(eval_schedule=2000, total_max_steps=100_000)

global_step=2009 | avg_return=18.000
global_step=4009 | avg_return=29.500
global_step=6011 | avg_return=27.667
global_step=8015 | avg_return=28.500
global_step=10011 | avg_return=27.400
global_step=12004 | avg_return=24.833
global_step=14001 | avg_return=25.857
global_step=16009 | avg_return=25.875
global_step=18003 | avg_return=26.556
global_step=20006 | avg_return=28.200
global_step=22023 | avg_return=34.300
global_step=24067 | avg_return=40.600
global_step=26046 | avg_return=44.200
global_step=28001 | avg_return=44.000
global_step=30026 | avg_return=45.100
global_step=32052 | avg_return=54.900
global_step=34045 | avg_return=62.300
global_step=36023 | avg_return=63.300
global_step=38011 | avg_return=65.700
global_step=40009 | avg_return=76.600
global_step=42054 | avg_return=87.700
global_step=44113 | avg_return=98.200
global_step=46132 | avg_return=110.500
global_step=48000 | avg_return=118.500
global_step=50023 | avg_return=121.900
global_step=52028 | avg_return=118.000
global_step=